# Qiskit QVE - SU(2) Matrix Model [Lambda = 4]

In [1]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
from qiskit.circuit.library import TwoLocal, EfficientSU2

import sys
sys.path.append('../../utility')
from vqe_run import *
from qc_ansatze import *

# Convert the Hamiltonian to strings of Pauli ops

In [2]:
# Load the H file already created using qiskit 0.28 from MBP 2015
file = open("../../utility/pauliH_L4_g1.0.txt", "r")
content = file.read()
#print(content)

In [3]:
data = content.split(',\n')
data[0] = data[0].split('\n')[1]
data[-1] = data[-1].split('\n')[0]

data_coeff = []
data_str = []
for i in range(len(data)):
    data_coeff.append(float(data[i].split('*')[0]))
    data_str.append(data[i].split('*')[1].split(' ')[1])
    
Hpauli = list(zip(data_str,data_coeff))
#Hpauli

In [4]:
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyEigensolver
H4q = SparsePauliOp.from_list(Hpauli)

solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],5)
E_exact

Exact Result of qubit hamiltonian: [3.52625444 4.78763673 4.78763673 4.78763673]


3.52625

# Circuit ansatze

In [5]:
#CIRCUIT ANSATZ
from qiskit.circuit.library import TwoLocal, EfficientSU2

In [6]:
reps=1
ansatz_0a = TL_ansatz(H4q.num_qubits, 'ry', 'crx', "circular", reps)
ansatz_0b = TL_ansatz(H4q.num_qubits, 'rz', 'crx', 'circular', reps)
ansatz_0c = TL_ansatz(H4q.num_qubits, ['ry','rz'], 'crx', 'circular', reps)
ansatz_0d = TL_ansatz(H4q.num_qubits,['ry','y'], 'crx', 'circular', reps)

ansatz_1a =  TL_ansatz(H4q.num_qubits, 'ry', 'crx', "full", reps)
ansatz_1b =  TL_ansatz(H4q.num_qubits, 'rz', 'crx', "full", reps)
ansatz_1c =  TL_ansatz(H4q.num_qubits, ['ry','rz'], 'crx', "full", reps)
ansatz_1d =  TL_ansatz(H4q.num_qubits, ['ry','y'], 'crx', "full", reps)

Circuit ansatz with 36 parameters
Circuit ansatz with 36 parameters
Circuit ansatz with 60 parameters
Circuit ansatz with 36 parameters
Circuit ansatz with 90 parameters
Circuit ansatz with 90 parameters
Circuit ansatz with 114 parameters
Circuit ansatz with 90 parameters


In [7]:
reps=1
ansatz_0a_su2 = ef_ansatz(H4q.num_qubits, 'ry',  "circular", reps)
ansatz_0b_su2 = ef_ansatz(H4q.num_qubits, 'rz',  'circular', reps)
ansatz_0c_su2 = ef_ansatz(H4q.num_qubits, ['ry','rz'], 'circular', reps)
ansatz_0d_su2 = ef_ansatz(H4q.num_qubits,['ry','y'], 'circular', reps)

ansatz_1a_su2 =  ef_ansatz(H4q.num_qubits, 'ry',  "full", reps)
ansatz_1b_su2 =  ef_ansatz(H4q.num_qubits, 'rz',  "full", reps)
ansatz_1c_su2 =  ef_ansatz(H4q.num_qubits, ['ry','rz'], "full", reps)
ansatz_1d_su2 =  ef_ansatz(H4q.num_qubits, ['ry','y'],  "full", reps)

Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 48 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 48 parameters
Circuit ansatz with 24 parameters


# Qiskit QVE: Noiseless simulator

https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.primitives.Estimator.html

https://qiskit-community.github.io/qiskit-algorithms/tutorials/03_vqe_simulation_with_noise.html

In [8]:
#VQE ALGORITHMS
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.utils import algorithm_globals 
from qiskit_aer.primitives import Estimator as AerEstimator

import warnings
warnings.filterwarnings("ignore")
seed = 170
iterations = 350
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.5f}")
    return result

# COBYLA: EffSU2

In [9]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0a_su2)
counts_0a_c = counts
values_0a_c = values

VQE result: 5.64314


In [10]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0b_su2)
counts_0b_c = counts
values_0b_c = values

VQE result: 3.89550


In [11]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0c_su2)
counts_0c_c = counts
values_0c_c = values

VQE result: 6.89548


In [12]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0d_su2)
counts_0d_c = counts
values_0d_c = values

VQE result: 6.65659


In [13]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1a_su2)
counts_1a_c = counts
values_1a_c = values

VQE result: 6.30709


In [14]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1b_su2)
counts_1b_c = counts
values_1b_c = values

VQE result: 3.89550


In [15]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1c_su2)
counts_1c_c = counts
values_1c_c = values

VQE result: 6.03170


In [16]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1d_su2)
counts_1d_c = counts
values_1d_c = values

VQE result: 4.30899


In [17]:
df1 = pd.DataFrame({'values_0a_su2': values_0a_c})
df2 = pd.DataFrame({'values_0b_su2': values_0b_c})
df3 = pd.DataFrame({'values_0c_su2': values_0c_c})
df4 = pd.DataFrame({'values_0d_su2': values_0d_c})
df5 = pd.DataFrame({'values_1a_su2': values_1a_c})
df6 = pd.DataFrame({'values_1b_su2': values_1b_c})
df7 = pd.DataFrame({'values_1c_su2': values_1c_c})
df8 = pd.DataFrame({'values_1d_su2': values_1d_c})
qve_l2_tl_cobyla = pd.concat([df1, df2, df3, df4, df5, df6, df7,df8], axis = 1)
qve_l2_tl_cobyla.to_csv('qve_l4_l=1.0_effsu2_cobyla.csv')
qve_l2_tl_cobyla

,values_0a_su2,values_0b_su2,values_0c_su2,values_0d_su2,values_1a_su2,values_1b_su2,values_1c_su2,values_1d_su2
0,18.238864,3.8955,17.320240,11.555572,17.384587,3.8955,18.618986,15.794905
1,20.669937,3.8955,17.220906,11.080493,19.133337,3.8955,19.249112,15.205208
2,18.498451,3.8955,17.570844,11.842126,18.241257,3.8955,20.157970,14.373984
3,18.587565,3.8955,18.051687,11.427519,19.196303,3.8955,17.168893,14.540694
4,16.846246,3.8955,17.761350,12.389467,17.546837,3.8955,17.472906,13.215987
...,...,...,...,...,...,...,...,...
345,NaN,NaN,7.079227,NaN,NaN,NaN,6.138003,4.315275
346,NaN,NaN,7.175832,NaN,NaN,NaN,6.230489,4.306989
347,NaN,NaN,7.005336,NaN,NaN,NaN,6.158951,4.308993
348,NaN,NaN,7.240691,NaN,NaN,NaN,6.227389,NaN


# COBYLA: TwoLocal

In [18]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0a)
counts_0a_c = counts
values_0a_c = values

VQE result: 3.34163


In [19]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0b)
counts_0b_c = counts
values_0b_c = values

VQE result: 3.89550


In [20]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0c)
counts_0c_c = counts
values_0c_c = values

VQE result: 6.02242


In [21]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0d)
counts_0d_c = counts
values_0d_c = values

VQE result: 4.94714


In [22]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1a)
counts_1a_c = counts
values_1a_c = values

VQE result: 3.56694


In [23]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1b)
counts_1b_c = counts
values_1b_c = values

VQE result: 3.89550


In [24]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1c)
counts_1c_c = counts
values_1c_c = values

VQE result: 4.60628


In [25]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1d)
counts_1d_c = counts
values_1d_c = values

VQE result: 3.55768


In [26]:
df1 = pd.DataFrame({'values_0a': values_0a_c})
df2 = pd.DataFrame({'values_0b': values_0b_c})
df3 = pd.DataFrame({'values_0c': values_0c_c})
df4 = pd.DataFrame({'values_0d': values_0d_c})
df5 = pd.DataFrame({'values_1a': values_1a_c})
df6 = pd.DataFrame({'values_1b': values_1b_c})
df7 = pd.DataFrame({'values_1c': values_1c_c})
df8 = pd.DataFrame({'values_1d': values_1d_c})
qve_l2_tl_cobyla = pd.concat([df1, df2, df3, df4, df5, df6, df7,df8], axis = 1)
qve_l2_tl_cobyla.to_csv('qve_l4_l=1.0_twolocal_cobyla.csv')
qve_l2_tl_cobyla

,values_0a,values_0b,values_0c,values_0d,values_1a,values_1b,values_1c,values_1d
0,11.564872,3.8955,21.186167,24.517564,21.359263,3.8955,21.771055,22.492601
1,12.218331,3.8955,20.885729,24.332071,21.612058,3.8955,20.371796,22.735122
2,12.828017,3.8955,21.735508,23.090605,20.662223,3.8955,22.680090,21.990585
3,11.488117,3.8955,21.018038,22.922545,20.983776,3.8955,20.557509,21.105480
4,12.955042,3.8955,20.923238,22.386801,21.886872,3.8955,18.353516,22.631992
...,...,...,...,...,...,...,...,...
345,3.403879,NaN,6.203914,4.962208,3.869883,3.8955,4.926763,3.644115
346,3.453957,NaN,6.293613,4.954381,3.596020,3.8955,5.217999,3.601088
347,3.408780,NaN,6.344275,4.977399,3.596992,3.8955,4.822669,3.567244
348,3.421897,NaN,6.579044,4.953290,3.648937,3.8955,4.802844,3.855675


# SPSA: EffSU2

In [27]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0a_su2)
counts_0a = counts
values_0a = values

VQE result: 4.22957


In [28]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0b_su2)
counts_0b = counts
values_0b = values

VQE result: 3.89550


In [29]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0c_su2)
counts_0c = counts
values_0c = values

VQE result: 8.01018


In [30]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0d_su2)
counts_0d = counts
values_0d = values

VQE result: 5.67670


In [31]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1a_su2)
counts_1a = counts
values_1a = values

VQE result: 4.19845


In [32]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1b_su2)
counts_1b = counts
values_1b = values

VQE result: 3.89550


In [33]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1c_su2)
counts_1c = counts
values_1c = values

VQE result: 3.78857


In [34]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1d_su2)
counts_1d = counts
values_1d = values

VQE result: 4.10703


In [35]:
# save the data to a dataframe 
list_l2 = {'values_0a_su2': values_0a, 'values_0b_su2': values_0b, 
        'values_0c_su2': values_0c, 'values_0d_su2': values_0d,
        'values_1a_su2': values_1a, 'values_1b_su2': values_1b, 
        'values_1c_su2': values_1c,'values_1d_su2': values_1d}
qve_l2_efsu2_spsa= pd.DataFrame(list_l2)
qve_l2_efsu2_spsa.to_csv('qve_l4_l=1.0_effsu2_spsa.csv')
qve_l2_efsu2_spsa

,values_0a_su2,values_0b_su2,values_0c_su2,values_0d_su2,values_1a_su2,values_1b_su2,values_1c_su2,values_1d_su2
0,17.219211,3.533716,21.684901,24.464840,20.049737,3.533716,19.565061,18.315780
1,18.554324,3.394166,20.325931,21.477723,22.803311,3.394166,20.243775,19.834667
2,18.507548,3.914714,21.235075,23.818007,23.023029,3.914714,19.455741,20.415484
3,18.742522,4.059289,20.311351,22.336063,20.982750,4.059289,19.580942,17.039113
4,18.731098,3.625436,20.972001,23.454739,20.997934,3.625436,18.143904,18.192763
...,...,...,...,...,...,...,...,...
746,4.611291,3.896976,8.115247,6.046926,4.507886,3.896976,3.858024,4.124817
747,4.237313,3.867935,8.239083,5.524822,4.407947,3.867935,4.242399,4.419094
748,4.337657,3.896976,7.771734,6.008281,4.498638,3.896976,3.843483,4.506984
749,4.550058,3.867935,8.356725,5.516375,4.354562,3.867935,3.892513,4.265940


# SPSA: TwoLocal

In [36]:
#3.18339
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0a)
counts_0a = counts
values_0a = values

VQE result: 5.26669


In [37]:
#3.17910
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0b)
counts_0b = counts
values_0b = values

VQE result: 3.89550


In [38]:
#3.62318
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0c)
counts_0c = counts
values_0c = values

VQE result: 3.55374


In [39]:
#3.13679
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0d)
counts_0d = counts
values_0d = values

VQE result: 4.83891


In [40]:
#3.16617
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1a)
counts_1a = counts
values_1a = values

VQE result: 3.39410


In [41]:
#3.17910
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1b)
counts_1b = counts
values_1b = values

VQE result: 3.89550


In [42]:
#4.10208
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1c)
counts_1c = counts
values_1c = values

VQE result: 3.92772


In [43]:
#4.26060
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1d)
counts_1d = counts
values_1d = values

VQE result: 4.97329


In [44]:
# save the data to a dataframe 
#don't reevaluate on a new run with just the new ansatze added
import pandas as pd
list_l2 = {'values_0a': values_0a, 'values_0b': values_0b, 
        'values_0c': values_0c, 'values_0d': values_0d,
        'values_1a': values_1a, 'values_1b': values_1b, 
        'values_1c': values_1c,'values_1d': values_1d}
qve_l2_tl_spsa= pd.DataFrame(list_l2)
qve_l2_tl_spsa.to_csv('qve_l4_l=1.0_twolocal_spsa.csv')
qve_l2_tl_spsa

,values_0a,values_0b,values_0c,values_0d,values_1a,values_1b,values_1c,values_1d
0,18.134355,3.533716,15.484540,15.794892,16.571443,3.533716,17.083664,20.583621
1,18.060469,3.394166,16.699371,17.747996,14.993756,3.394166,15.293454,20.277663
2,16.369102,3.914714,15.453940,18.353901,16.826947,3.914714,15.737907,20.491461
3,20.249254,4.059289,15.527856,15.492400,14.934541,4.059289,15.288599,19.519453
4,19.537438,3.625436,15.562093,18.895430,16.344939,3.625436,15.841501,18.440559
...,...,...,...,...,...,...,...,...
746,5.530037,3.896976,3.887153,4.838997,3.690201,3.896976,4.248749,5.219852
747,4.817171,3.867935,3.663343,5.116660,3.868509,3.867935,4.184915,4.554735
748,5.612278,3.896976,3.879501,4.843624,3.720984,3.896976,4.346683,5.093859
749,5.393280,3.867935,3.690134,4.956622,3.500223,3.867935,3.655088,4.975369
